In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource
import datetime
from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf

from pyflink.datastream import StreamExecutionEnvironment
from pyflink.common.typeinfo import Types
from pyflink.datastream.functions import RuntimeContext, MapFunction
from pyflink.table import StreamTableEnvironment
from pyflink.datastream.state import ValueStateDescriptor
# create a batch TableEnvironment
#env_settings = EnvironmentSettings.in_streaming_mode()
#table_env = TableEnvironment.create(env_settings)
#table_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.0/lib/*.jar")

In [2]:
env = StreamExecutionEnvironment.get_execution_environment()
table_env = StreamTableEnvironment.create(stream_execution_environment=env)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [3]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom

source_ddl = """
            CREATE TABLE invoices (
                   InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING,
                   WATERMARK FOR InvoiceDate AS InvoiceDate - INTERVAL '1' SECOND

                    ) WITH (
                      'connector' = 'filesystem',          
                      'path' = 'file:///home/krish/flink/data/ecomm/ecommerce-small.csv', 
                      'format' = 'csv'
                    )"""


table_env.execute_sql(source_ddl) 

invoices = table_env.from_path("invoices")
##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()



Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` TIMESTAMP(3) *ROWTIME*,
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING,
  WATERMARK FOR `InvoiceDate`: TIMESTAMP(3) AS `InvoiceDate` - INTERVAL '1' SECOND
)


In [4]:
invoices.fetch(3).execute().print()

+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+--------------------------------+--------------------------------+--------------------------------+
| op |                      InvoiceNo |                      StockCode |                    Description |                       Quantity |             InvoiceDate |                      UnitPrice |                     CustomerID |                        Country |
+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+--------------------------------+--------------------------------+--------------------------------+
| +I |                         536365 |                         85123A | WHITE HANGING HEART T-LIGHT... |                            6.0 | 2010-12-01 08:26:00.000 |                           2.55 |           

In [5]:
schema = invoices.get_schema()

ds = table_env.to_append_stream(
    table_env.from_path('invoices'),
    Types.ROW([Types.STRING(), Types.STRING(), Types.STRING(), Types.DOUBLE(), 
                Types.SQL_TIMESTAMP(), Types.DOUBLE(),Types.STRING(), Types.STRING()]))

In [6]:
ds.print()

In [7]:
env.execute()

4> +I[536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850,United Kingdom]
4> +I[536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom]
4> +I[536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850,United Kingdom]
4> +I[536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom]
4> +I[536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom]
4> +I[536365,22752,SET 7 BABUSHKA NESTING BOXES,2.0,2010-12-01 08:26:00,7.65,17850,United Kingdom]
4> +I[536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,4.25,17850,United Kingdom]
4> +I[536366,22633,HAND WARMER UNION JACK,6.0,2010-12-01 08:28:00,1.85,17850,United Kingdom]
4> +I[536366,22632,HAND WARMER RED POLKA DOT,6.0,2010-12-01 08:28:00,1.85,17850,United Kingdom]
4> +I[536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,2010-12-01 08:34:00,1.69,13047,United Kingdom]


In [8]:
class MyMapFunction(MapFunction):

    def open(self, runtime_context: RuntimeContext):
        print("MyMapFunction open")
        state_desc = ValueStateDescriptor('cnt', Types.LONG())
        #Define value state
        self.cnt_state = runtime_context.get_state(state_desc)
        print ("Count state is ", self.cnt_state)
        

    def map(self, value):
        print("Value is ", value, value[0], value[2])
        cnt = self.cnt_state.value()
        print("MyMapFunction map", cnt)
        if cnt is None:
            cnt = 0

        new_cnt = cnt + 1
        self.cnt_state.update(new_cnt)
        return value[0], new_cnt

In [9]:

#3. Define execution logic
ds2 = ds.key_by(lambda a: a[0]) \
       .map(MyMapFunction(), output_type=Types.TUPLE([Types.LONG(), Types.LONG()]))


In [10]:
ds2.print()
env.execute()

MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7fc707e883d0>
MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7fc707ea4760>
Value is  Row(f0='536367', f1='84879', f2='ASSORTED COLOUR BIRD ORNAMENT', f3=32.0, f4=datetime.datetime(2010, 12, 1, 8, 34), f5=1.69, f6='13047', f7='United Kingdom') 536367 ASSORTED COLOUR BIRD ORNAMENT
Value is  Row(f0='536366', f1='22633', f2='HAND WARMER UNION JACK', f3=6.0, f4=datetime.datetime(2010, 12, 1, 8, 28), f5=1.85, f6='17850', f7='United Kingdom') 536366 HAND WARMER UNION JACK
MyMapFunction map None
MyMapFunction map None
MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7fc707e493d0>
Value is  Row(f0='536365', f1='85123A', f2='WHITE HANGING HEART T-LIGHT HOLDER', f3=6.0, f4=datetime.datetime(2010, 12, 1, 8, 26), f5=2.55, f6='17850', f7='United Kingdom') 536365 WHITE HANG

Traceback (most recent call last):
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/runners/worker/sdk_worker.py", line 289, in _execute
    response = task()
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/runners/worker/sdk_worker.py", line 362, in <lambda>
    lambda: self.create_worker().do_instruction(request), request)
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/runners/worker/sdk_worker.py", line 606, in do_instruction
    return getattr(self, request_type)(
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/runners/worker/sdk_worker.py", line 644, in process_bundle
    bundle_processor.process_bundle(instruction_id))
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/runners/worker/bundle_processor.py", line 999, in process_bundle
    input_op_by_transform_id[element.transform_id].process_encoded(
  File "/home/krish/miniconda3/lib/python3.8/site-packages/apache_beam/

Py4JJavaError: An error occurred while calling o0.execute.
: org.apache.flink.runtime.client.JobExecutionException: Job execution failed.
	at org.apache.flink.runtime.jobmaster.JobResult.toJobExecutionResult(JobResult.java:144)
	at org.apache.flink.runtime.minicluster.MiniClusterJobClient.lambda$getJobExecutionResult$3(MiniClusterJobClient.java:141)
	at java.util.concurrent.CompletableFuture.uniApply(CompletableFuture.java:616)
	at java.util.concurrent.CompletableFuture$UniApply.tryFire(CompletableFuture.java:591)
	at java.util.concurrent.CompletableFuture.postComplete(CompletableFuture.java:488)
	at java.util.concurrent.CompletableFuture.complete(CompletableFuture.java:1975)
	at org.apache.flink.runtime.rpc.akka.AkkaInvocationHandler.lambda$invokeRpc$1(AkkaInvocationHandler.java:259)
	at java.util.concurrent.CompletableFuture.uniWhenComplete(CompletableFuture.java:774)
	at java.util.concurrent.CompletableFuture$UniWhenComplete.tryFire(CompletableFuture.java:750)
	at java.util.concurre

In [ ]:

class MyKeyedProcessFunction(KeyedProcessFunction):

    def open(self, runtime_context: RuntimeContext):
        state_desc = MapStateDescriptor('map', Types.LONG(), Types.LONG())
        self.state = runtime_context.get_map_state(state_desc)

    def process_element(self, value, ctx: 'KeyedProcessFunction.Context'):
        if not self.state.contains(value[0]):
            result = value[1]
            self.state.put(value[0], result)
        else:
            existing = self.state.get(value[0])
            if existing <= 10:
                result = value[1] + existing
                self.state.put(value[0], result)
            else:
                result = existing
        yield result
